In [79]:
import os
import re
import requests

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.parse import urlparse

from tqdm import tqdm


In [127]:
def checkURL(requested_url):
    if not urlparse(requested_url).scheme:
        requested_url = "https://" + requested_url
    return requested_url


def requestAndParse(requested_url):
    requested_url = checkURL(requested_url)
    try:
        # define headers to be provided for request authentication
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                        'AppleWebKit/537.11 (KHTML, like Gecko) '
                        'Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
        request_obj = Request(url=requested_url, headers=headers)
        page_html = urlopen(request_obj)
        page_soup = BeautifulSoup(page_html, "html.parser")
        return page_soup

    except Exception as e:
        print(e)

def crawlImageURL(soupObject):
    collection = []
    soup = soupObject
    for img in soup.find_all('img'):
        collection.append(img.attrs['src'])
    collection = set(collection) #Use set to remove duplicate values.
    return list(collection)

def makeDirectory(parent_dir, directoryName):
    path = os.path.join(parent_dir, directoryName)
    if not path: 
        os.mkdir(path)
        print('{} is created!!!'.format(directoryName))
    else:
        print('{} is exist!!!'.format(path))

def downloadImgWithURL(urlCollection, category):

    for url in urlCollection:

        indexes = re.split('/|\?', url)
        for idx in indexes:
            if re.search('\d\.(jpg|png|jpeg)', idx):
                filename = idx
                
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            response.raw.decode_content = True
            open('./Data/' + filename, 'wb').write(response.content)
            # if os.path.isfile('./Data/' + category):
            #     open('./Data/' + filename, 'wb').write(response.content)
            # else:
            #     makeDirectory('./Data/', category)
            #     open('./Data/' + category + '/' + filename, 'wb').write(response.content)


In [129]:
url_resources = ['https://www.pexels.com/search/', 'https://www.freeimages.com/search/']

animal_list = ['cat', 'lion', 'leopard', 'tiger', 'jaguar', 'sphynx', 
                'dog', 'wolf', 'husky', 'corgi', 'pug']

# soupObject = requestAndParse('https://www.freeimages.com/search/cat')
# imgCollection = crawlImageURL(soupObject)
# imgCollection

for animal in tqdm(animal_list):
    soupObject = requestAndParse(url_resources[0] + animal + '/')
    imgCollection = crawlImageURL(soupObject)
    downloadImgWithURL(imgCollection, animal)


100%|██████████| 11/11 [06:36<00:00, 36.06s/it]


In [123]:
soupObject = requestAndParse('https://www.pexels.com/search/cat')
imgCollection = crawlImageURL(soupObject)
imgCollection

HTTP Error 308: Permanent Redirect


AttributeError: 'NoneType' object has no attribute 'find_all'